In [2]:
import pandas as pd

path_to_data = "..//data//"
file_path = "daily_almaty_6oct20.xlsx"

sheets_dict = pd.read_excel(path_to_data+file_path, sheet_name=None)

print("Sheet names:", sheets_dict.keys())

Sheet names: dict_keys(['sensors', 'data', 'coil_comp', 'march_comp'])


In [4]:
sheets_dict['sensors'].to_csv(path_to_data+'sensors.csv')

## prepare data sheet

In [88]:
import datetime
import locale
locale.setlocale(locale.LC_TIME, 'en_EN')

'en_EN'

In [ ]:
df = sheets_dict['data'].copy()
df = df.rename(columns={'Row Labels': 'date'})

In [98]:
for i, val in enumerate(df['date'].values):
    if type(val) != str:
        df.loc[i, "date"] = val.strftime('%d-%b')

In [99]:
start_year = 2017

df['year'] = start_year
for i in range(1, len(df)):
    # If the date goes "backward" (e.g., Oct follows Mar), increment the year
    if pd.to_datetime(df['date'][i] + f"-{df['year'][i-1]}") < pd.to_datetime(df['date'][i-1] + f"-{df['year'][i-1]}"):
        df.loc[i:, 'year'] += 1

# Combine day, month, and year to form a full date
df['full_date'] = pd.to_datetime(df['date'] + "-" + df['year'].astype(str))

df = df.drop(columns=['date', 'year'])

C:\Users\77019\AppData\Local\Temp\ipykernel_9400\3222366651.py:10: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



In [100]:
columns = [df.columns[-1]]+df.columns[:-1].tolist()

In [101]:
df[columns].to_csv(path_to_data+"pm_data.csv")

## creating test set

In [57]:
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [65]:
nan_percentage = df.isnull().mean() * 100

threshold = 95  
filtered_df = df.loc[:, nan_percentage <= threshold]

In [ ]:
nan_counts = df.set_index("full_date").isnull().groupby("full_date").sum()

# Split features into chunks of 5
features = nan_counts.columns
chunks = [features[i:i + 5] for i in range(0, len(features), 5)]

# Plot each chunk
for chunk_idx, chunk in enumerate(chunks):
    fig = go.Figure()
    for feature in chunk:
        fig.add_trace(
            go.Scatter(
                x=nan_counts.index,
                y=nan_counts[feature],
                mode='lines+markers',
                name=feature
            )
        )
    fig.update_layout(
        title=f"NaN Counts (Chunk {chunk_idx + 1})",
        xaxis_title="Date",
        yaxis_title="NaN Count",
        legend_title="Features"
    )
    fig.show()

In [66]:
for column in filtered_df.columns:
    if column != "full_date":
        fig = go.Figure()
        fig.add_trace(
            go.Scatter(
                x=df['full_date'],
                y=df[column],
                mode='lines+markers',
                name=column,
                connectgaps=False  # Ensures gaps for NaN values
            )
        )
        fig.update_layout(
            title=f"Values of {column} Over Time",
            xaxis_title="Date",
            yaxis_title="Value",
            legend_title="Feature"
        )
        fig.show()

# train test split

In [ ]:
import plotly.graph_objects as go

df['nan_percentage'] = df.isnull().mean(axis=1) * 100

# Create a Plotly line plot
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df['full_date'],  # Time series on the x-axis
        y=df['nan_percentage'],  # Percentage of NaNs on the y-axis
        mode='lines+markers',
        name='NaN Percentage',
        line=dict(color='blue'),
        marker=dict(size=6)
    )
)

# Update layout for better visualization
fig.update_layout(
    title="Percentage of NaN Values Across Columns for Each Row",
    xaxis_title="Time",
    yaxis_title="Percentage of NaNs (%)",
    template="plotly_white",
)

fig.show()

In [ ]:
import plotly.graph_objects as go

filtered_df['nan_percentage'] = filtered_df.isnull().mean(axis=1) * 100

# Create a Plotly line plot
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=filtered_df['full_date'],  # Time series on the x-axis
        y=filtered_df['nan_percentage'],  # Percentage of NaNs on the y-axis
        mode='lines+markers',
        name='NaN Percentage',
        line=dict(color='blue'),
        marker=dict(size=6)
    )
)

# Update layout for better visualization
fig.update_layout(
    title="Percentage of NaN Values Across Columns for Each Row",
    xaxis_title="Time",
    yaxis_title="Percentage of NaNs (%)",
    template="plotly_white",
)

fig.show()

In [ ]:
oot_start_date = '2020-03-01'

oot_df = df.loc[df['full_date'] >= oot_start_date]
train_df = df.loc[df['full_date'] < oot_start_date]

In [ ]:
oot_df.to_csv(path_to_data+"raw_oot.csv")
train_df.to_csv(path_to_data+"raw_train.csv")